In [2]:
import os
import sys

current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..')) 
if project_root not in sys.path:
    sys.path.append(project_root)

from backend.src.logger import logging
from backend.src.exception import CustomException
from langchain_tavily import TavilySearch

tool = TavilySearch(
            max_results=5,
            topic="general",
            api_key=os.getenv("TAVILY_API_KEY")
        )

async def fetch_materials(topic, subject, standard) -> dict:
        """Node A: Fetches external resources using Tavily and returns a structured list."""
        try:
            logging.info("Async Node: Fetching study materials...")
            
            topic = topic
            subject = subject
            standard = standard

            query = f"Study materials, tutorials, and youtube videos for {topic} in {subject} for grade/standard {standard}"
            
            response = await tool.ainvoke(query)

            links = []
            
            # Standardize response to a list regardless of API format
            results_list = []
            if isinstance(response, list):
                results_list = response
            elif isinstance(response, dict) and "results" in response:
                results_list = response["results"]
            
            # Extract only URL and Title
            for item in results_list:
                if isinstance(item, dict):
                    links.append({
                        "url": item.get("url", "No URL"),
                        "title": item.get("title", item.get("content", "Resource")[:50])
                    })

            # Returns List[Dict] exactly as requested
            return {"raw_study_links": links}

        except Exception as e:
            logging.error(f"Error in fetching materials: {str(e)}")
            raise CustomException(e, sys)

In [3]:
topic = "Atomic Structure"
subject = "Physics"
standard = 11

response = await fetch_materials(topic, subject, standard)

In [4]:
response["raw_study_links"]

[{'url': 'https://www.youtube.com/playlist?list=PLrFJvH84tTqtkNiOue6ddjseWZwdpouNI',
  'title': 'Grade 11 Atomic Structure'},
 {'url': 'https://www.youtube.com/watch?v=EMDrb2LqL7E',
  'title': 'Atomic Structure: Protons, Electrons & Neutrons'},
 {'url': 'https://www.youtube.com/watch?v=aP96pTYX3xE',
  'title': 'Grade 11 Physics - Atoms and Isotopes'},
 {'url': 'https://www.youtube.com/watch?v=OH-aSu-rWgk',
  'title': 'Chemistry - Atomic Structure - EXPLAINED!'},
 {'url': 'https://www.youtube.com/watch?v=sCkasELDYls',
  'title': 'Atomic Combinations grade 11 :Lewis diagram # 1'}]

In [5]:
from backend.src.agents.assistant_agent import AssistantAgent

topic = "Atomic Structure"
subject = "Physics"
standard = 11

# 1. Instantiate the agent directly
agent = AssistantAgent()

# 2. Use 'await' directly (works only in Jupyter/IPython)
assistant_response = await agent.run(
    topic=topic,
    subject=subject,
    standard=standard
)

In [7]:
print(assistant_response)

### System Prompt for Lesson Planning Agent

#### [CONTEXT]
Create a 35-50 lesson curriculum for the topic "Atomic Structure" in Physics for Standard 11 students. The curriculum should be based on the provided data, including the student guide, resources, and raw search links.

#### [OBJECTIVES]
The objectives of this curriculum are to:
- Analyze the difficulty level of the topic "Atomic Structure" and break it down into sequential lessons.
- Ensure the pace of the lessons suits a Standard 11 student.
- Provide a comprehensive understanding of the topic, including its concepts, formulas, and applications.

#### [CONSTRAINTS]
The constraints for this curriculum are:
- The curriculum should consist of 35-50 lessons.
- The lessons should be designed to accommodate a Standard 11 student's pace and understanding.
- The curriculum should include a mix of theoretical and practical lessons to engage students and promote learning.
- The lessons should be aligned with the provided student guide 